![](https://wordpress.accuweather.com/wp-content/uploads/2020/03/CoronavirusAnimation2.gif)

# Introduction

**The first case of the COVID-19 pandemic in India was reported on 30 January 2020, originating from China. As of 4 May 2020, the Ministry of Health and Family Welfare have confirmed a total of 42,533 cases, 11,707 recoveries (including 1 migration) and 1373 deaths in the country. Experts suggest the number of infections could be much higher as India's testing rates are among the lowest in the world. The infection rate of COVID-19 in India is reported to be 1.7, significantly lower than in the worst affected countries.**

**The outbreak has been declared an epidemic in more than a dozen states and union territories, where provisions of the Epidemic Diseases Act, 1897 have been invoked, and educational institutions and many commercial establishments have been shut down. India has suspended all tourist visas, as a majority of the confirmed cases were linked to other countries.**

**On 22 March 2020, India observed a 14-hour voluntary public curfew at the instance of the prime minister Narendra Modi. The government followed it up with lockdowns in 75 districts where COVID-19 cases had occurred as well as all major cities. Further, on 24 March, the prime minister ordered a nationwide lockdown for 21 days, affecting the entire 1.3 billion population of India. On 14 April, the prime minister extended the ongoing nationwide lockdown till 3 May.**

**Michael Ryan, chief executive director of the World Health Organisation's health emergencies programme, said that India had "tremendous capacity" to deal with the coronavirus outbreak and, as the second most populous country, will have enormous impact on the world's ability to deal with it. Other commentators worried about the economic devastation caused by the lockdown, which has huge effects on informal workers, micro and small enterprises, farmers and the self-employed, who are left with no livelihood in the absence of transportation and access to markets.**

**Observers state that the lockdown has slowed the growth rate of the pandemic by 6 April to a rate of doubling every 6 days, and, by 18 April, to a rate of doubling every 8 days.**

**The Oxford COVID-19 Government Response Tracker (OxCGRT), in its report based on data from 73 countries, reports that the Indian Government has responded more stringently than other countries in tackling the pandemic. It noted the government's swift action, emergency policy making emergency investment in healthcare, fiscal measures, investment in vaccine research and active response to the situation, and scored India with a "100" for its strictness.**

**As of 27 April the states of Goa, Sikkim, Nagaland, Arunachal Pradesh, Manipur and Tripura have been declared as "Covid-19 Free" with zero active cases by the Government of India. More states are expected to join this list in a few weeks.**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.integrate import odeint

from plotly.offline import iplot, init_notebook_mode
import math
import bokeh 
import matplotlib.pyplot as plt
import plotly.express as px
from urllib.request import urlopen
import json
from dateutil import parser
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file
from bokeh.layouts import row, column
from bokeh.resources import INLINE
from bokeh.io import output_notebook
from bokeh.models import Span
import warnings
warnings.filterwarnings("ignore")
output_notebook(resources=INLINE)

# Load Data

In [ ]:
virus_data = pd.read_csv('/kaggle/input/novel-corona-virus-2019-dataset/covid_19_data.csv')
virus_data.head(5)

In [ ]:
covid_India_cases = pd.read_csv('../input/covid19-in-india/covid_19_india.csv')
covid_India_cases.rename(columns={'State/UnionTerritory': 'State', 'Cured': 'Recovered', 'Confirmed': 'Confirmed'}, inplace=True)

#statewise_cases = pd.DataFrame(covid_India_cases.groupby(['State'])['Confirmed', 'Deaths', 'Recovered'].max().reset_index())
covid_India_cases.head(5)

In [ ]:
country_codes = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')
country_codes = country_codes.drop('GDP (BILLIONS)', 1)
country_codes.rename(columns={'COUNTRY': 'Country', 'CODE': 'Code'}, inplace=True)

# Visualization

In [ ]:
virus_data = pd.read_csv('/kaggle/input/novel-corona-virus-2019-dataset/covid_19_data.csv')

prev_index = 0
first_time = False
tmp = 0


for i, row in virus_data.iterrows():

    if(virus_data.loc[i,'SNo'] < 1342 and virus_data.loc[i,'Province/State']=='Hubei'):
        if(first_time):
            tmp = virus_data.loc[i,'Confirmed']
            prev_index = i
            virus_data.loc[i,'Confirmed'] = virus_data.loc[i,'Confirmed'] + 593
            first_time = False
        else:
            increment = virus_data.loc[i,'Confirmed'] - tmp
            tmp = virus_data.loc[i,'Confirmed']
            virus_data.loc[i,'Confirmed'] = virus_data.loc[prev_index,'Confirmed'] + increment + 593
            prev_index = i
    

virus_data.rename(columns={'Country/Region': 'Country', 'ObservationDate': 'Date'}, inplace=True)
virus_data = virus_data.fillna('unknow')
virus_data['Country'] = virus_data['Country'].str.replace('US','United States')
virus_data['Country'] = virus_data['Country'].str.replace('UK','United Kingdom') 
virus_data['Country'] = virus_data['Country'].str.replace('Mainland China','China')
virus_data['Country'] = virus_data['Country'].str.replace('South Korea','Korea, South')
virus_data['Country'] = virus_data['Country'].str.replace('North Korea','Korea, North')
virus_data['Country'] = virus_data['Country'].str.replace('Macau','China')
virus_data['Country'] = virus_data['Country'].str.replace('Ivory Coast','Cote d\'Ivoire')
virus_data = pd.merge(virus_data,country_codes,on=['Country'])
#virus_data.head()
#print(len(virus_data))

In [ ]:
covid_India_cases = covid_India_cases.fillna('unknow')
top_country = covid_India_cases.loc[covid_India_cases['Date'] == covid_India_cases['Date'].iloc[-1]]
top_country = top_country.groupby(['State'])['Confirmed'].sum().reset_index()
top_country = top_country.sort_values('Confirmed', ascending=False)
top_country = top_country[:30]
top_country_codes = top_country['State']
top_country_codes = list(top_country_codes)

#countries = virus_data.loc[virus_data['Country'] in top_country_codes]
countries = covid_India_cases[covid_India_cases['State'].isin(top_country_codes)]
countries_day = countries.groupby(['Date','State'])['Confirmed','Deaths','Recovered'].sum().reset_index()


exponential_line_x = []
exponential_line_y = []
for i in range(16):
    exponential_line_x.append(i)
    exponential_line_y.append(i)
    
################################    Maharashtra    #################
Maharashtra = countries_day.loc[countries_day['State']=='Maharashtra']
Maharashtra=Maharashtra.sort_values('Confirmed',ascending=True)

new_confirmed_cases_Maharashtra = []
new_confirmed_cases_Maharashtra.append( list(Maharashtra['Confirmed'])[0] - list(Maharashtra['Deaths'])[0] 
                           - list(Maharashtra['Recovered'])[0] )

for i in range(1,len(Maharashtra)):

    new_confirmed_cases_Maharashtra.append( list(Maharashtra['Confirmed'])[i] - 
                                     list(Maharashtra['Deaths'])[i] - 
                                     list(Maharashtra['Recovered'])[i])
    
#######################   Gujarat   ############
Gujarat = countries_day.loc[countries_day['State']=='Gujarat']
Gujarat=Gujarat.sort_values('Confirmed',ascending=True)

new_confirmed_cases_Gujarat = []
new_confirmed_cases_Gujarat.append( list(Gujarat['Confirmed'])[0] - list(Gujarat['Deaths'])[0] 
                           - list(Gujarat['Recovered'])[0] )

for i in range(1,len(Gujarat)):
    
    new_confirmed_cases_Gujarat.append( list(Gujarat['Confirmed'])[i] - 
                                  list(Gujarat['Deaths'])[i] - 
                                  list(Gujarat['Recovered'])[i])
    
###########################    Delhi   ##################3
Delhi = countries_day.loc[countries_day['State']=='Delhi']
Delhi=Delhi.sort_values('Confirmed',ascending=True)

new_confirmed_cases_Delhi = []
new_confirmed_cases_Delhi.append( list(Delhi['Confirmed'])[0] - list(Delhi['Deaths'])[0] 
                           - list(Delhi['Recovered'])[0] )

for i in range(1,len(Delhi)):
    
    new_confirmed_cases_Delhi.append( list(Delhi['Confirmed'])[i] - 
                                     list(Delhi['Deaths'])[i] - 
                                    list(Delhi['Recovered'])[i])
    
#############################    Madhya Pradesh    ################3
Madhya_Pradesh = countries_day.loc[countries_day['State']=='Madhya Pradesh']
Madhya_Pradesh=Madhya_Pradesh.sort_values('Confirmed',ascending=True)

new_confirmed_cases_Madhya_Pradesh = []
new_confirmed_cases_Madhya_Pradesh.append( list(Madhya_Pradesh['Confirmed'])[0] - list(Madhya_Pradesh['Deaths'])[0] 
                           - list(Madhya_Pradesh['Recovered'])[0] )

for i in range(1,len(Madhya_Pradesh)):
    
    new_confirmed_cases_Madhya_Pradesh.append( list(Madhya_Pradesh['Confirmed'])[i] - 
                                     list(Madhya_Pradesh['Deaths'])[i] - 
                                    list(Madhya_Pradesh['Recovered'])[i])
    
################################   Rajasthan  ##########
Rajasthan = countries_day.loc[countries_day['State']=='Rajasthan']
Rajasthan=Rajasthan.sort_values('Confirmed',ascending=True)

new_confirmed_cases_Rajasthan = []
new_confirmed_cases_Rajasthan.append( list(Rajasthan['Confirmed'])[0] - list(Rajasthan['Deaths'])[0] 
                           - list(Rajasthan['Recovered'])[0] )

for i in range(1,len(Rajasthan)):
    
    new_confirmed_cases_Rajasthan.append( list(Rajasthan['Confirmed'])[i] - 
                                     list(Rajasthan['Deaths'])[i] - 
                                    list(Rajasthan['Recovered'])[i])
    
#################################    Uttar Pradesh   ##############
Uttar_Pradesh = countries_day.loc[countries_day['State']=='Uttar Pradesh']
Uttar_Pradesh=Uttar_Pradesh.sort_values('Confirmed',ascending=True)

new_confirmed_cases_Uttar_Pradesh = []
new_confirmed_cases_Uttar_Pradesh.append( list(Uttar_Pradesh['Confirmed'])[0] - list(Uttar_Pradesh['Deaths'])[0] 
                           - list(Uttar_Pradesh['Recovered'])[0] )

for i in range(1,len(Uttar_Pradesh)):
    
    new_confirmed_cases_Uttar_Pradesh.append( list(Uttar_Pradesh['Confirmed'])[i] - 
                                     list(Uttar_Pradesh['Deaths'])[i] - 
                                    list(Uttar_Pradesh['Recovered'])[i])
    
#####################################  Tamil Nadu  ############
Tamil_Nadu = countries_day.loc[countries_day['State']=='Tamil Nadu']
Tamil_Nadu=Tamil_Nadu.sort_values('Confirmed',ascending=True)

new_confirmed_cases_Tamil_Nadu = []
new_confirmed_cases_Tamil_Nadu.append( list(Tamil_Nadu['Confirmed'])[0] - list(Tamil_Nadu['Deaths'])[0] 
                           - list(Tamil_Nadu['Recovered'])[0] )

for i in range(1,len(Tamil_Nadu)):
    
    new_confirmed_cases_Tamil_Nadu.append( list(Tamil_Nadu['Confirmed'])[i] - 
                                     list(Tamil_Nadu['Deaths'])[i] - 
                                    list(Tamil_Nadu['Recovered'])[i])
######################################## Andhra Pradesh ##############
Andhra_Pradesh = countries_day.loc[countries_day['State']=='Andhra Pradesh']
Andhra_Pradesh=Andhra_Pradesh.sort_values('Confirmed',ascending=True)

new_confirmed_cases_Andhra_Pradesh = []
new_confirmed_cases_Andhra_Pradesh.append( list(Andhra_Pradesh['Confirmed'])[0] - list(Andhra_Pradesh['Deaths'])[0] 
                           - list(Andhra_Pradesh['Recovered'])[0] )

for i in range(1,len(Andhra_Pradesh)):
    
    new_confirmed_cases_Andhra_Pradesh.append( list(Andhra_Pradesh['Confirmed'])[i] - 
                                     list(Andhra_Pradesh['Deaths'])[i] - 
                                    list(Andhra_Pradesh['Recovered'])[i])

########################################Telengana#####################

Telengana = countries_day.loc[countries_day['State']=='Telengana']
Telengana=Telengana.sort_values('Confirmed',ascending=True)

new_confirmed_cases_Telengana = []
new_confirmed_cases_Telengana.append( list(Telengana['Confirmed'])[0] - list(Telengana['Deaths'])[0] 
                           - list(Telengana['Recovered'])[0] )

for i in range(1,len(Telengana)):
    
    new_confirmed_cases_Telengana.append( list(Telengana['Confirmed'])[i] - 
                                     list(Telengana['Deaths'])[i] - 
                                    list(Telengana['Recovered'])[i])


##########################################  West Bengal #####################33
West_Bengal = countries_day.loc[countries_day['State']=='West Bengal']
West_Bengal=West_Bengal.sort_values('Confirmed',ascending=True)

new_confirmed_cases_West_Bengal = []
new_confirmed_cases_West_Bengal.append( list(West_Bengal['Confirmed'])[0] - list(West_Bengal['Deaths'])[0] 
                           - list(West_Bengal['Recovered'])[0] )

for i in range(1,len(West_Bengal)):
    
    new_confirmed_cases_West_Bengal.append( list(West_Bengal['Confirmed'])[i] - 
                                     list(West_Bengal['Deaths'])[i] - 
                                    list(West_Bengal['Recovered'])[i])
############################################ 
p1 = figure(plot_width=800, plot_height=550, title="Trajectory of Covid-19")
p1.grid.grid_line_alpha=0.3
p1.ygrid.band_fill_color = "olive"
p1.ygrid.band_fill_alpha = 0.1
p1.xaxis.axis_label = 'Total number of detected cases (Log scale)'
p1.yaxis.axis_label = 'New confirmed cases (Log scale)'

p1.line(exponential_line_x, exponential_line_y, line_dash="4 4", line_width=0.5)

p1.line(np.log(list(Maharashtra['Confirmed'])), np.log(new_confirmed_cases_Maharashtra), color='#DBAE23', 
        legend_label='Maharashtra', line_width=1)
p1.circle(np.log(list(Maharashtra['Confirmed'])[-1]), np.log(new_confirmed_cases_Maharashtra[-1]), fill_color="white", size=5)

p1.line(np.log(list(Gujarat['Confirmed'])), np.log(new_confirmed_cases_Gujarat), color='#3EC358', 
        legend_label='Gujarat', line_width=1)
p1.circle(np.log(list(Gujarat['Confirmed'])[-1]), np.log(new_confirmed_cases_Gujarat[-1]), fill_color="white", size=5)

p1.line(np.log(list(Delhi['Confirmed'])), np.log(new_confirmed_cases_Delhi), color='#C3893E', 
       legend_label='Delhi', line_width=1)
p1.circle(np.log(list(Delhi['Confirmed'])[-1]), np.log(new_confirmed_cases_Delhi[-1]), fill_color="white", size=5)


p1.line(np.log(list(Madhya_Pradesh['Confirmed'])), np.log(new_confirmed_cases_Madhya_Pradesh), color='#3E4CC3', 
        legend_label='Madhya Pradesh', line_width=1)
p1.circle(np.log(list(Madhya_Pradesh['Confirmed'])[-1]), np.log(new_confirmed_cases_Madhya_Pradesh[-1]), fill_color="white", size=5)

p1.line(np.log(list(Rajasthan['Confirmed'])), np.log(new_confirmed_cases_Rajasthan), color='#F54138', 
        legend_label='Rajasthan', line_width=1)
p1.circle(np.log(list(Rajasthan['Confirmed'])[-1]), np.log(new_confirmed_cases_Rajasthan[-1]), fill_color="white", size=5)

p1.line(np.log(list(Uttar_Pradesh['Confirmed'])), np.log(new_confirmed_cases_Uttar_Pradesh), color='#23BCDB', 
        legend_label='Uttar Pradesh', line_width=1)
p1.circle(np.log(list(Uttar_Pradesh['Confirmed'])[-1]), np.log(new_confirmed_cases_Uttar_Pradesh[-1]), fill_color="white", size=5)

p1.line(np.log(list(Tamil_Nadu['Confirmed'])), np.log(new_confirmed_cases_Tamil_Nadu), color='#010A0C', 
        legend_label='Tamil Nadu', line_width=1)
p1.circle(np.log(list(Tamil_Nadu['Confirmed'])[-1]), np.log(new_confirmed_cases_Tamil_Nadu[-1]), fill_color="white", size=5)

p1.line(np.log(list(Andhra_Pradesh['Confirmed'])), np.log(new_confirmed_cases_Andhra_Pradesh), color='#bf40bf', 
        legend_label='Andhra Pradesh', line_width=1)
p1.circle(np.log(list(Andhra_Pradesh['Confirmed'])[-1]), np.log(new_confirmed_cases_Andhra_Pradesh[-1]), fill_color="white", size=5)

p1.line(np.log(list(Telengana['Confirmed'])), np.log(new_confirmed_cases_Telengana), color='lime', 
        legend_label='Telengana', line_width=1)
p1.circle(np.log(list(Telengana['Confirmed'])[-1]), np.log(new_confirmed_cases_Telengana[-1]), fill_color="white", size=5)


p1.line(np.log(list(West_Bengal['Confirmed'])), np.log(new_confirmed_cases_West_Bengal), color='#0000ff', 
        legend_label='West Bengal', line_width=1)
p1.circle(np.log(list(West_Bengal['Confirmed'])[-1]), np.log(new_confirmed_cases_West_Bengal[-1]), fill_color="white", size=5)



p1.legend.location = "bottom_right"

output_file("coronavirus.html", title="coronavirus.py")

show(p1)

In [ ]:
p1 = figure(plot_width=800, plot_height=550, title="Trajectory of Covid-19")
p1.grid.grid_line_alpha=0.3
p1.ygrid.band_fill_color = "olive"
p1.ygrid.band_fill_alpha = 0.1
p1.xaxis.axis_label = 'Total number of detected cases (Log scale)'
p1.yaxis.axis_label = 'New confirmed cases (Log scale)'

p1.line(exponential_line_x, exponential_line_y, line_dash="4 4", line_width=0.5)

p1.line(np.log(list(Maharashtra['Confirmed'])), np.log(new_confirmed_cases_Maharashtra), color='#DBAE23', 
        legend_label='Maharashtra', line_width=1)
p1.circle(np.log(list(Maharashtra['Confirmed'])[-1]), np.log(new_confirmed_cases_Maharashtra[-1]), fill_color="white", size=5)

p1.line(np.log(list(West_Bengal['Confirmed'])), np.log(new_confirmed_cases_West_Bengal), color='Blue', 
        legend_label='West_Bengal', line_width=1)
p1.circle(np.log(list(West_Bengal['Confirmed'])[-1]), np.log(new_confirmed_cases_West_Bengal[-1]), fill_color="white", size=5)



p1.legend.location = "bottom_right"

output_file("coronavirus.html", title="coronavirus.py")

show(p1)

In [ ]:
import IPython
IPython.display.HTML('<div class="flourish-embed flourish-bar-chart-race" data-src="visualisation/1977187" data-url="https://flo.uri.sh/visualisation/1977187/embed"><script src="https://public.flourish.studio/resources/embed.js"></script></div>')

In [ ]:
df = pd.read_csv('../input/covid19-in-india/covid_19_india.csv')
data = df.copy()
data['Date'] = data['Date'].apply(pd.to_datetime)
data.drop(['Sno', 'Time'],axis=1,inplace=True)

# collect present data
from datetime import date
data_apr = data[data['Date'] > pd.Timestamp(date(2020,4,12))]

# prepaing data state wise
state_cases = data_apr.groupby('State/UnionTerritory')['Confirmed','Deaths','Cured'].max().reset_index()
state_cases['Active'] = state_cases['Confirmed'] - (state_cases['Deaths']- state_cases['Cured'])
state_cases["Death Rate (per 100)"] = np.round(100*state_cases["Deaths"]/state_cases["Confirmed"],2)
state_cases["Cure Rate (per 100)"] = np.round(100*state_cases["Cured"]/state_cases["Confirmed"],2)

In [ ]:
state_details = pd.pivot_table(df, values=['Confirmed','Deaths','Cured'], index='State/UnionTerritory', aggfunc='max')
state_details['Recovery Rate'] = round(state_details['Cured'] / state_details['Confirmed'],2)
state_details['Deaths']['Madhya Pradesh#']=119

state_details['Deaths']=state_details['Deaths'].astype(np.float32)
#state_details=state_details.reset_index()
state_details['Active']=state_details['Confirmed']-state_details['Cured']-state_details['Deaths']

state_details['Death Rate'] = round(state_details['Deaths'] /state_details['Confirmed'], 2)
state_details = state_details.sort_values(by='Confirmed', ascending= False)
#state_details.style.background_gradient(cmap='PuBuGn')
state_details.style.bar(subset=['Confirmed'], color='#ff781c')\
                    .bar(subset=['Cured'], color='green')\
                    .bar(subset=['Deaths'], color='red')\
                    .bar(subset=['Active'], color='#cd4f39')\
                    .bar(subset=['Recovery Rate'], color='lime')\
                    .bar(subset=['Death Rate'], color='gray')

# Report Profilling

In [ ]:
from pandas_profiling import ProfileReport 
report = ProfileReport(state_cases)
report

# Test

# Tree Plot

In [ ]:
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=False)
labs = pd.read_csv("../input/covid19-in-india/ICMRTestingLabs.csv")
fig = px.treemap(labs, path=['state','city'],
                  color='city', hover_data=['lab','address'],
                  color_continuous_scale='reds')
fig.show()

# State

In [ ]:
plt.figure(figsize=(35, 15))
plt.title('Labs Across State')
labs.state.value_counts().plot.bar();

# City

In [ ]:
plt.figure(figsize=(35, 15))
plt.title('Labs Across City')
labs.city.value_counts().plot.bar();

In [ ]:
state_details=state_details.reset_index()
fig = px.sunburst(state_details, path=['State/UnionTerritory'], values='Active',
                  color='Active', hover_data=['Confirmed','Cured','Deaths'],
                  color_continuous_scale='PuBuGn')
fig.show()

# StateWise analysis of Positive cases

In [ ]:
india_map=state_details
from plotly.subplots import make_subplots
import plotly.graph_objects as go
l=list(india_map["State/UnionTerritory"])
fig = make_subplots(rows=11, cols=3,subplot_titles=l,specs=[[{'type':'domain'}, {'type':'domain'},{'type':'domain'}],[{'type':'domain'}, {'type':'domain'},{'type':'domain'}],[{'type':'domain'}, {'type':'domain'},{'type':'domain'}],[{'type':'domain'}, {'type':'domain'},{'type':'domain'}],[{'type':'domain'}, {'type':'domain'},{'type':'domain'}],[{'type':'domain'}, {'type':'domain'},{'type':'domain'}],[{'type':'domain'}, {'type':'domain'},{'type':'domain'}],[{'type':'domain'}, {'type':'domain'},{'type':'domain'}],[{'type':'domain'}, {'type':'domain'},{'type':'domain'}],[{'type':'domain'}, {'type':'domain'},{'type':'domain'}],[{'type':'domain'}, {'type':'domain'},{'type':'domain'}]])
a=1
b=1

for i in l:
    
    
    temp_df=india_map[india_map["State/UnionTerritory"]==i]
    values=[int(temp_df["Deaths"]),int(temp_df["Cured"]),int(temp_df["Active"])]
    labels=["Cured","Deaths","Active"]
 
    #annot.append(dict(text=i,font_size=10, showarrow=False))
    
    fig.add_trace(go.Pie(labels=labels, textposition="inside",values=values, name=i),a, b)
    
    if b==3 and a<11:
        a=a+1
   
      
    if b+1>3:
        b=1
    else:
        b=b+1
   
    fig.update_traces(hole=.4)

fig.update_layout(
    
    height=1900,width=1000
)
fig.update(layout_title_text='StateWise analysis of Positive cases')


#fig = go.Figure(fig)
fig.show()
#iplot(fig)  

In [ ]:
import datetime
india_covid_19 = pd.read_csv('../input/covid19-in-india/covid_19_india.csv')
india_covid_19['Date'] = pd.to_datetime(india_covid_19['Date'])

cumulative_df = india_covid_19.groupby(["State/UnionTerritory", "Date"])["Confirmed", "Deaths", "Cured"].sum().reset_index()
cumulative_df["Date"] = pd.to_datetime(cumulative_df["Date"] , format="%d/%m/%y").dt.date
cumulative_df = cumulative_df.sort_values(by="Date").reset_index(drop=True)
start_date = datetime.date(2020, 3, 10)
cumulative_df = cumulative_df[cumulative_df["Date"]>=start_date]
cumulative_df["Date"] = cumulative_df["Date"].astype(str)

fig = px.scatter(cumulative_df, x="Confirmed", y="Deaths", animation_frame="Date", animation_group="State/UnionTerritory",
           size="Confirmed", color="State/UnionTerritory", hover_name="State/UnionTerritory",
           log_x=False, size_max=55, range_x=[0,15000], range_y=[-20,800])

layout = go.Layout(
    title=go.layout.Title(
        text="Changes in number of confirmed & death cases over time in India states",
        x=0.5
    ),
    font=dict(size=14),
    xaxis_title = "Total number of confirmed cases",
    yaxis_title = "Total number of death cases"
)

fig.update_layout(layout)

fig.show()



In [ ]:
import datetime
india_covid_19 = pd.read_csv('../input/covid19-in-india/covid_19_india.csv')
india_covid_19['Date'] = pd.to_datetime(india_covid_19['Date'])

cumulative_df = india_covid_19.groupby(["State/UnionTerritory", "Date"])["Confirmed", "Deaths", "Cured"].sum().reset_index()
cumulative_df["Date"] = pd.to_datetime(cumulative_df["Date"] , format="%d/%m/%y").dt.date
cumulative_df = cumulative_df.sort_values(by="Date").reset_index(drop=True)
start_date = datetime.date(2020, 3, 10)
cumulative_df = cumulative_df[cumulative_df["Date"]>=start_date]
cumulative_df["Date"] = cumulative_df["Date"].astype(str)

fig = px.scatter(cumulative_df, x="Confirmed", y="Cured", animation_frame="Date", animation_group="State/UnionTerritory",
           size="Confirmed", color="State/UnionTerritory", hover_name="State/UnionTerritory",
           log_x=False, size_max=55, range_x=[0,15000], range_y=[-20,2500])

layout = go.Layout(
    title=go.layout.Title(
        text="Changes in number of confirmed & recovered cases over time in India states",
        x=0.5
    ),
    font=dict(size=14),
    xaxis_title = "Total number of confirmed cases",
    yaxis_title = "Total number of recovered cases"
)

fig.update_layout(layout)

fig.show()


In [ ]:
import datetime
india_covid_19 = pd.read_csv('../input/covid19-in-india/covid_19_india.csv')
india_covid_19['Date'] = pd.to_datetime(india_covid_19['Date'])

cumulative_df = india_covid_19.groupby(["State/UnionTerritory", "Date"])["Confirmed", "Deaths", "Cured"].sum().reset_index()
cumulative_df["Date"] = pd.to_datetime(cumulative_df["Date"] , format="%d/%m/%y").dt.date
cumulative_df = cumulative_df.sort_values(by="Date").reset_index(drop=True)
start_date = datetime.date(2020, 3, 10)
cumulative_df = cumulative_df[cumulative_df["Date"]>=start_date]
cumulative_df["Date"] = cumulative_df["Date"].astype(str)

fig = px.scatter(cumulative_df, x="Cured", y="Deaths", animation_frame="Date", animation_group="State/UnionTerritory",
           size="Confirmed", color="State/UnionTerritory", hover_name="State/UnionTerritory",
           log_x=False, size_max=55, range_x=[0,2500], range_y=[-20,800])

layout = go.Layout(
    title=go.layout.Title(
        text="Changes in number of recovered & death cases over time in India states",
        x=0.5
    ),
    font=dict(size=14),
    xaxis_title = "Total number of recovered cases",
    yaxis_title = "Total number of death cases"
)

fig.update_layout(layout)

fig.show()


# Containment Zones

In [ ]:
zone=pd.read_csv('/kaggle/input/covid-19-india-zone-classification/lockdownindiawarningzones.csv')

In [ ]:
zone.style.set_properties(**{'background-color': 'black',
                           'color': 'lawngreen',
                           'border-color': 'white'})

![](https://images.livemint.com/img/2020/04/19/original/Chart1a_1587287717356.png)

In [ ]:
def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{columnNames[i]}')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()
    
def plotCorrelationMatrix(df, graphWidth):
    filename = df.dataframeName
    df = df.dropna('columns') # drop columns with NaN
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    if df.shape[1] < 2:
        print(f'No correlation plots shown: The number of non-NaN or constant columns ({df.shape[1]}) is less than 2')
        return
    corr = df.corr()
    plt.figure(num=None, figsize=(graphWidth, graphWidth), dpi=80, facecolor='w', edgecolor='k')
    corrMat = plt.matshow(corr, fignum = 1)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.gca().xaxis.tick_bottom()
    plt.colorbar(corrMat)
    plt.title(f'Correlation Matrix for {filename}', fontsize=15)
    plt.show()

def plotScatterMatrix(df, plotSize, textSize):
    df = df.select_dtypes(include =[np.number]) # keep only numerical columns
    # Remove rows and columns that would lead to df being singular
    df = df.dropna('columns')
    df = df[[col for col in df if df[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    columnNames = list(df)
    if len(columnNames) > 10: # reduce the number of columns for matrix inversion of kernel density plots
        columnNames = columnNames[:10]
    df = df[columnNames]
    ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
    corrs = df.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=textSize)
    plt.suptitle('Scatter and Density Plot')
    plt.show()

In [ ]:
plotPerColumnDistribution(zone, 10, 5)

In [ ]:
fig = px.treemap(zone, path=["State",'Zone','District'],
                  color='Zone',
                 color_discrete_map={'(?)':'black','Green Zone':'#00FF00', 'Red Zone':'#ff0000', 'Orange Zone':'#FFA500'},
                 hover_data=['Zone'])
fig.show()

![](https://www.hindustantimes.com/images/app-images/2020/4/DelhiMetro_pg7_gfx.jpg)

Reference: 
1. https://www.kaggle.com/soham1024/covid-19-india-visualization-forecasting#Containment-Zones-in-India
2. https://www.kaggle.com/vanshjatana/coronavirus-attack-in-india

# State Wise:

In [ ]:
state_details

# To be continued....

# Hope you guys find this notebook useful.
# If you like this notebook, please upvote.***👍
# That will keep me motivated :)